In [1]:
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.optimizers import RMSprop
from keras.utils import to_categorical

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tnrange

Using TensorFlow backend.
/home/erwan/.local/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [3]:
side_size = 508
print('system image data format:', K.image_data_format())
input_img = Input(shape=(side_size, side_size, 3))  # adapt this if using `channels_first` image data format
print('image shape', input_img.shape)

system image data format: channels_last
image shape (?, 508, 508, 3)


In [4]:
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

generator = Model(input_img, decoded)
generator_model = generator.compile(optimizer='adadelta', loss='binary_crossentropy')
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 508, 508, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 508, 508, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 254, 254, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 254, 254, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 127, 127, 32)      18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 64, 32)        0         
__________

In [5]:
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(64)(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
x = Dense(1)(x)

discriminator = Model(input_img, Activation('sigmoid')(x))
discriminator.compile(loss='binary_crossentropy', 
                            optimizer=RMSprop(lr=0.0008, clipvalue=1.0, decay=6e-8), 
                            metrics=['accuracy'])
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 508, 508, 3)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 508, 508, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 254, 254, 32)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 254, 254, 64)      18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 127, 127, 32)      18464     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 64, 64, 32)        0         
__________

In [6]:
optimizer = RMSprop(lr=0.0004, clipvalue=1.0, decay=3e-8)

adversarial_model = Sequential()
adversarial_model.add(generator)
adversarial_model.add(discriminator)

adversarial_model.compile(loss='binary_crossentropy', 
                          optimizer=optimizer,
                          metrics=['accuracy'])

In [7]:
batch_size = 4

train_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    'impress_dataset',
    target_size=(side_size, side_size),
    batch_size=batch_size,
    class_mode='input')

Found 999 images belonging to 1 classes.


In [8]:
def train(train_steps=50):
    discriminator_losses = []
    adversarial_losses = []
    sample_images = []
    
    for i in tnrange(train_steps):
        print(i)
        # Select a random sample from the training data and the labels
        images_train = train_generator[i % batch_size][0]
        
        # Create noise in range -1 to 1 and random labels as input for the generator to generate the fake images
        # noise_input, _ = create_generator_noise(batch_size)
        # Instead of noise let us try real images even if that is not protocol
        images_fake = generator.predict(images_train)
        
        # Create input by concatenate both real and fake images and assigning the respective labels
        # labels_fake = np.zeros(shape=(batch_size, classes+1))
        # labels_fake[:,-1] = 1
        
        input_data   = np.concatenate((images_train, images_fake))
        input_labels = np.concatenate([np.ones(shape=(batch_size)), np.zeros(shape=(batch_size))])

        discriminator_loss = discriminator.train_on_batch(input_data, input_labels)
        
        # Train the adversarial model to generate better images
        noise_input, noise_labels = create_generator_noise(batch_size)
        noise_labels = np.concatenate((noise_labels, np.zeros(shape=(batch_size, 1))), axis=1)
        
        adversarial_loss = adversarial_model.train_on_batch(images_train, np.ones(shape=(batch_size)))
        
        discriminator_losses.append(discriminator_loss)
        adversarial_losses.append(adversarial_loss)
        
        if i % 100 == 0:
            labels = [1]
            fake_images = generator.predict(images_train[0])
            sample_images.append(fake_images[0])
    
    return discriminator_losses, adversarial_losses, sample_images

In [9]:
discriminator_losses, adversarial_losses, sample_images  = train()

0


ResourceExhaustedError: OOM when allocating tensor with shape[8,32,508,508] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv2d_10/convolution}} = Conv2D[T=DT_FLOAT, _class=["loc:@training/RMSprop/gradients/conv2d_10/convolution_grad/Conv2DBackpropFilter"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training/RMSprop/gradients/conv2d_10/convolution_grad/Conv2DBackpropFilter-0-TransposeNHWCToNCHW-LayoutOptimizer, conv2d_10/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node metrics_1/acc/Mean_1/_173}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge

In [ ]:
batch_input = train_generator[0][0].astype(int)
batch_result = 255 - autoencoder.predict(train_generator[0][0]).astype(int)*255
for input_img, result_img in zip(batch_input, batch_result):
    _, axarr = plt.subplots(1,2)
    axarr[0].imshow(input_img)
    axarr[0].axis('off')
    axarr[1].imshow(result_img)
    axarr[1].axis('off')